# Imports

In [1]:
import duckdb
import altair as alt
import polars as pl
import yfinance as yf


# DataSet

* [Share Prices in the United Kingdom (SPPUKA)](https://fred.stlouisfed.org/series/SPPUKA)

This dataset was downloaded from FRED (Federal Reserve Bank of St. Louis).
Please see the series notes as stated on the FRED website.

> This series was constructed by the Bank of England as part of the Three Centuries of Macroeconomic Data project by combining data from a number of academic and official sources. For more information, please refer to the Three Centuries spreadsheet at https://www.bankofengland.co.uk/statistics/research-datasets. Users are advised to check the underlying assumptions behind this series in the relevant worksheets of the spreadsheet. In many cases alternative assumptions might be appropriate. Users are permitted to reproduce this series in their own work as it represents Bank calculations and manipulations of underlying series that are the copyright of the Bank of England provided that underlying sources are cited appropriately. For appropriate citation please see the Three Centuries spreadsheet for guidance and a list of the underlying sources.


In [3]:
duckdb.sql(f"""SELECT * FROM SPPUKA.csv""").pl().plot.line(x="observation_date", y=alt.Y("SPPUKA", scale=alt.Scale(type="log")))

alt.Chart(...)

# Panic Cycle

In [ ]:
tooltips = ["cycle_period", "price", "samples"]

In [ ]:
base_year = 1749
end_year = 1965
chart = duckdb.sql(f"""
SELECT
    (YEAR(observation_date) - {base_year}) % 54 cycle_period,
    AVG(SPPUKA) price,
    COUNT(SPPUKA) samples,
FROM SPPUKA.csv
WHERE
    {base_year} < YEAR(observation_date)
    AND YEAR(observation_date) < {end_year}
GROUP BY ALL
""").pl().plot.line(x="cycle_period", y=alt.Y("price", scale=alt.Scale(type="log")), tooltip=tooltips)


In [ ]:
source = pl.DataFrame({
    'cycle_period': [0, 16, 16+18, 16+18+20],
})

# Chart with vertical lines at each date
rules = alt.Chart(source).mark_rule(color='red', strokeWidth=2).encode(
    x='cycle_period'
)
chart + rules

# High Prices

In [10]:
tooltips = ["cycle_period", "price", "samples"]
cycle_length = 27
base_year = 1729
end_year = 1999
data = duckdb.sql(f"""
SELECT
    (YEAR(observation_date) - {base_year}) % {cycle_length} cycle_period,
    AVG(SPPUKA) price,
    COUNT(SPPUKA) samples,
FROM SPPUKA.csv
WHERE
    {base_year} < YEAR(observation_date)
    AND YEAR(observation_date) < {end_year}
GROUP BY ALL
""").pl()
chart = data.plot.line(x="cycle_period", y=alt.Y("price", scale=alt.Scale(type="log")), tooltip=tooltips)
chart

alt.Chart(...)

In [5]:
highlights = alt.Chart(
    pl.DataFrame({
        'cycle_period': [0, 8, 8+9, 8+9+10],
    })
).mark_rule(color='red', strokeWidth=2).encode(
    x='cycle_period'
)
chart + highlights


alt.LayerChart(...)

# Low Prices

In [ ]:
tooltips = ["cycle_period", "price", "samples"]
cycle_length = 27
base_year = 1735
end_year = 2005
chart = duckdb.sql(f"""
SELECT
    (YEAR(observation_date) - {base_year}) % {cycle_length} cycle_period,
    AVG(SPPUKA) price,
    COUNT(SPPUKA) samples,
FROM SPPUKA.csv
WHERE
    {base_year} < YEAR(observation_date)
    AND YEAR(observation_date) < {end_year}
GROUP BY ALL
""").pl().plot.line(x="cycle_period", y=alt.Y("price", scale=alt.Scale(type="log")), tooltip=tooltips)
chart


In [ ]:
import altair as alt
import polars as pl

highlights = alt.Chart(
    pl.DataFrame({
        'cycle_period': [0, 7, 7+11, 7+11+9],
    })
).mark_rule(color='red', strokeWidth=2).encode(
    x='cycle_period'
)
chart + highlights

# Winrate

In [11]:
data.filter(
    pl.col("cycle_period").is_in([0, 8]),
)

cycle_period,price,samples
i64,f64,i64
8,43.582,10
0,38.293333,9
